In [141]:
from SRC.foursquare import exploreForesquare, searchForesquare
from SRC.geofunc import *
from SRC.mongofunc import connectMongoColl
from geopy import distance

from haversine import haversine
# harvesine (lat,long) --> dist in km.

import pandas as pd
from pymongo import MongoClient
import numpy as np

# Geo_Apis
### For the best value, let's see the situation of the rest of the filters
---

In [133]:
coord = [37.772323,-122.214897]

## 1 | PARTY

In [21]:
party = searchForesquare(ID="party",limit=15,distance=500,latitude=37.772323,longitude=-122.214897)

In [22]:
dfparty = pd.DataFrame(party["response"]["venues"])
dfparty

,id,name,location,categories,referralId,hasPerk
0,4e7e9edf8231243aa62dff89,Party Spot,"{'lat': 37.776357, 'lng': -122.216319, 'labele...","[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",v-1581364090,False
1,4f40ad70e4b01e824ddddb6f,12th Street Lounge,"{'address': '3700 E 12th St Fl 2', 'lat': 37.7...","[{'id': '4bf58dd8d48988d1d8941735', 'name': 'G...",v-1581364090,False
2,4ae687dff964a52013a721e3,La Frontera Nightclub,"{'address': '4481 International Blvd', 'lat': ...","[{'id': '4bf58dd8d48988d118941735', 'name': 'D...",v-1581364090,False
3,4eaa7989e3000d1f05738cbb,The Bricks,"{'lat': 37.773972, 'lng': -122.22118701487776,...","[{'id': '4bf58dd8d48988d1d6941735', 'name': 'S...",v-1581364090,False
4,4d42444a33268cfaa0465701,Mel's Loft,"{'address': '4001 San Leandro St', 'crossStree...","[{'id': '4bf58dd8d48988d1d4941735', 'name': 'S...",v-1581364090,False


In [72]:
loc_party = []
for i in dfparty.location:
    loc_party.append([float(i['lat']),float(i['lng'])])

In [73]:
loc_party

[[37.776357, -122.216319],
 [37.774036189028635, -122.22128921920071],
 [37.771166, -122.213862],
 [37.773972, -122.22118701487776],
 [37.771209000000006, -122.21972099999999]]

In [74]:
names = []
for i in dfparty.name:
    names.append(i)

names
    

['Party Spot',
 '12th Street Lounge',
 'La Frontera Nightclub',
 'The Bricks',
 "Mel's Loft"]

In [75]:
part =  dict(zip(names,loc_party))

In [81]:
dfparty = pd.DataFrame(part)
dfparty = dfparty.T.reset_index().rename(columns={"index":"name",0: "latitude", 1: "longitude"})
dfparty["location"] = dfparty[["latitude","longitude"]].apply(lambda x:asGeoJSON(x.latitude,x.longitude), axis=1)
dfparty = dfparty.drop(columns=["latitude","longitude"])
display(dfparty)

,name,location
0,Party Spot,"{'type': 'Point', 'coordinates': [-122.216319,..."
1,12th Street Lounge,"{'type': 'Point', 'coordinates': [-122.2212892..."
2,La Frontera Nightclub,"{'type': 'Point', 'coordinates': [-122.213862,..."
3,The Bricks,"{'type': 'Point', 'coordinates': [-122.2211870..."
4,Mel's Loft,"{'type': 'Point', 'coordinates': [-122.2197209..."


## 2 | Starbucks

In [123]:
dfst = pd.read_csv("INPUT/starbucks.csv")
dfst.head()

,Brand,Store Number,Name,Ownership Type,Facility ID,Features - Products,Features - Service,Features - Stations,Food Region,Venue Type,...,Street Line 1,Street Line 2,City,State,Zip,Country,Coordinates,Latitude,Longitude,Insert Date
0,Starbucks,11854,Epping Main St,Company Owned,18228,"Lunch, Oven-warmed Food, Reserve Amenity","Starbucks Card Mobile, Wireless Hotspot",Drive-Through,9999,Unknown,...,96 Calef Highway,NaN,Epping,NH,03042-2224,US,"(43.031873, -71.073203)",43.031873,-71.073203,06/22/2012 06:17:10 PM
1,Starbucks,79420,Stop & Shop-Exeter #204,Licensed,9956,NaN,Wireless Hotspot,NaN,9999,Unknown,...,139 Portsmouth Ave,NaN,Exeter,NH,03833-2105,US,"(42.989763, -70.932044)",42.989763,-70.932044,06/22/2012 06:17:10 PM
2,Starbucks,3872,Target Greenland T-2530,Licensed,10933,NaN,Starbucks Card Mobile,NaN,9999,Unknown,...,1450 Greenland Rd,NaN,Greenland,NH,03840-2438,US,"(43.04871, -70.816345)",43.048710,-70.816345,06/22/2012 06:17:10 PM
3,Starbucks,13448,"Seabrook, Lafayette Rd",Company Owned,16980,"Lunch, Oven-warmed Food, Reserve Amenity, Sire...","Starbucks Card Mobile, Wireless Hotspot",Drive-Through,9999,Unknown,...,271 Lafayette Rd,NaN,Seabrook,NH,03874-4512,US,"(42.882128, -70.86808)",42.882128,-70.868080,06/22/2012 06:17:10 PM
4,Starbucks,10325,"Portsmouth, Woodbury Avenue",Company Owned,17269,"Lunch, Oven-warmed Food, Reserve Amenity","Starbucks Card Mobile, Wireless Hotspot",Drive-Through,9999,Unknown,...,1855 Woodbury Avenue,NaN,Portsmouth,NH,03801-3228,US,"(43.092318, -70.792327)",43.092318,-70.792327,06/22/2012 06:17:10 PM


In [125]:
dfst["location"] = dfst[["Latitude","Longitude"]].apply(lambda x:asGeoJSON(x.Latitude,x.Longitude), axis=1)
dfst = dfst.loc[:, ['Brand', 'Name', "State",'location']]
dfst.head()

,Brand,Name,State,location
0,Starbucks,Epping Main St,NH,"{'type': 'Point', 'coordinates': [-71.073203, ..."
1,Starbucks,Stop & Shop-Exeter #204,NH,"{'type': 'Point', 'coordinates': [-70.93204399..."
2,Starbucks,Target Greenland T-2530,NH,"{'type': 'Point', 'coordinates': [-70.816345, ..."
3,Starbucks,"Seabrook, Lafayette Rd",NH,"{'type': 'Point', 'coordinates': [-70.86808, 4..."
4,Starbucks,"Portsmouth, Woodbury Avenue",NH,"{'type': 'Point', 'coordinates': [-70.792327, ..."


In [135]:
val=[]
for i in dfst.location:
    if haversine(coord,i["coordinates"]) < 50:
        val.append("YES")
    else:
        val.append("NO")
        
dfst.loc[:, "near_point"] = val


In [139]:
dfst.to_json("OUTPUT/Starbucks.json", orient="records")
# $ mongoimport --db starbucks --collection starbucks --jsonArray --drop OUTPUT/Starbucks.json

In [160]:
db, collst = connectMongoColl('starbucks','starbucks')

In [154]:
result = collst.aggregate([
    {
        '$geoNear': {
            'near': {
                'type': 'Point', 
                'coordinates': [
                    -122.214897, 37.772323
                ]
            }, 
            'distanceField': 'dist.calculated', 
            'minDistance': 2, 
            'spherical': True
        }
    }
])

In [157]:
# COMO SE HACE ESTO!!!!!

In [169]:
db, collst_dist = connectMongoColl('starbucks','starbucks_dist')

In [172]:
filt = {"Brand":1,"State":1,"location":1,"dist":1}
starb = list(collst_dist.find({"dist.calculated":{"$lte":10000}},filt))
df = pd.DataFrame(starb)
df

""


In [173]:
#POR QUÉ NO FUNCIONA!